In [8]:
import os
import shutil
import re
import subprocess
import urllib
import zipfile
import requests


"""
Scrapes and installs chromium from linux mint 21.3(virginia) packages site.
Link: http://packages.linuxmint.com/pool/upstream/c/chromium/
Scrapes and installs chromedriver from Chrome for Testing page.
Link: https://googlechromelabs.github.io/chrome-for-testing/
"""

class CantGetLatestChromiumVersionError(Exception):
    """Happens when regex failed"""

class ChromiumInstallationFailedException(Exception):
    """
    Happens when deb package not installed
    Check the downloaded chroumium deb file
    """

class CantGetChromeDriverError(Exception):
    """Happens when regex failed"""

main_url = "http://packages.linuxmint.com/pool/upstream/c/chromium/"
work_dir = "/content"

def get_chromium_latest_version() -> str:
    # A request to packages.linuxmint.com for getting latest version of chromium
    # e.g. "chromium_121.0.6167.160~linuxmint1+virginia_amd64.deb"
    r = requests.get(main_url)
    if r.status_code != 200:
        raise Exception("status_code code not 200!")
    text = r.text

    # Find latest version
    pattern = '<a\shref="(chromium_[^"]+linuxmint1%2Bvirginia_amd64.deb)'
    latest_version_search = re.search(pattern, text)
    if latest_version_search:
        latest_version = latest_version_search.group(1)
    else:
        raise CantGetLatestChromiumVersionError("Failed to get latest chromium version!")
    return latest_version

def install_chromium(latest_version: str, deb_file: str, quiet: bool):
    # Full url of deb file
    url = f"{main_url}{latest_version}"

    # Download deb file
    if quiet:
        command = f"wget -q -O {work_dir}/{deb_file} {url}"
    else:
        command = f"wget -O {work_dir}/{deb_file} {url}"
    print(f"Downloading: {deb_file}")
    # os.system(command)
    !$command

    # Install deb file
    if quiet:
        command = f"apt-get install {work_dir}/{deb_file} >> apt.log"
    else:
        command = f"apt-get install {work_dir}/{deb_file}"
    print(f"Installing: {deb_file}")
    # os.system(command)
    !$command

def check_chromium_installation(deb_file: str):
    try:
        subprocess.call(["chromium"])
        print("Chromium installation successfull.\n")
        # If installation successfull we can remove deb file
        # Delete deb file from disk
        os.remove(f"{work_dir}/{deb_file}")
    except FileNotFoundError:
        raise ChromiumInstallationFailedException("Chromium Installation Failed!")

def get_chromedriver_url(deb_file: str) -> str:
    # Get content of crhomedriver page
    url = "https://googlechromelabs.github.io/chrome-for-testing/"
    r = requests.get(url)
    if r.status_code != 200:
        raise Exception("status_code code not 200!")
    text = r.text

    # Get chromium version from deb file's name
    version_number = deb_file.split("chromium_")[-1].split(".")[0]

    # Example: https://edgedl.me.gvt1.com/edgedl/chrome/chrome-for-testing/121.0.6167.85/linux64/chromedriver-linux64.zip
    pattern = f'https://[^<]+/{version_number}[^<]+/linux64/chromedriver-linux64.zip'
    # Find latest version
    chromedriver_url_search = re.search(pattern, text)
    #if chromedriver_url_search:
    #    chromedriver_url = chromedriver_url_search.group()
    #    return chromedriver_url
    #else:
    #    raise CantGetChromeDriverError("Failed to get chromedriver!")

def install_chromedriver(deb_file: str, quiet: bool):
    url = get_chromedriver_url(deb_file)
    file_name = url.split("/")[-1]
    # Download chromedriver
    chromedriver_zip = f"{work_dir}/{file_name}"
    if quiet:
        command = f"wget -q -O {chromedriver_zip} {url}"
    else:
        command = f"wget -O {chromedriver_zip} {url}"
    print(f"Downloading: {file_name}")
    # os.system(command)
    !$command

    # Extract chromedriver from zip
    with zipfile.ZipFile(chromedriver_zip) as zpf:
        zpf.extract(member="chromedriver-linux64/chromedriver", path=work_dir)

    # Remove chromedriver-linux64.zip file
    os.remove(chromedriver_zip)

    # Move extracted chromedriver binary file to /usr/bin directory
    source = f"{work_dir}/chromedriver-linux64/chromedriver"
    destination = "/usr/bin/chromedriver"
    os.rename(source, destination)

    # Make chromedriver binary executable
    os.system(f"chmod +x {destination}")

    # Remove empty chromedriver-linux64 folder
    shutil.rmtree(f"{work_dir}/chromedriver-linux64")

    print("Chromedriver installed")

def install_selenium_package(quiet: bool):
    if quiet:
        !pip install selenium -qq >> pip.log
    else:
        !pip install selenium

def main(quiet: bool):
    # Get the latest version of chromium from linux mint packages site
    latest_version = get_chromium_latest_version()
    # Name of the deb file
    deb_file = urllib.parse.unquote(latest_version, "utf-8")
    # Download and install chromium for ubuntu 22.04
    install_chromium(latest_version, deb_file, quiet)
    # Check if installation succesfull
    check_chromium_installation(deb_file)
    # Install chromedriver
    #install_chromedriver(deb_file, quiet)
    # Finally install selenium package
    install_selenium_package(quiet)

if __name__ == '__main__':
    quiet = True # verboseness of wget and apt
    main(quiet)

Downloading: chromium_130.0.6723.116~linuxmint1+virginia_amd64.deb
Installing: chromium_130.0.6723.116~linuxmint1+virginia_amd64.deb
Chromium installation successfull.



In [2]:
!pip install selenium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.7/481.7 kB 26.0 MB/s eta 0:00:00


In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select, WebDriverWait
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import NoAlertPresentException
from selenium.webdriver.support import expected_conditions as EC
import time
import random

def random_delay(min_delay=5, max_delay=10):
    time.sleep(random.uniform(min_delay, max_delay))

user_agent = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/12.0.3 Safari/605.1.15"
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument(f"user-agent={user_agent}")
chrome_driver_path = '/usr/bin/chromedriver'
driver = webdriver.Chrome(service=Service(chrome_driver_path), options=chrome_options)
driver.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", {
  "source": """
    Object.defineProperty(navigator, 'webdriver', {
      get: () => undefined
    })
  """
})
driver.get('https://lvr.land.moi.gov.tw/')
random_delay()

driver.switch_to.frame(0)
#print(driver.page_source)
city = Select(driver.find_element(By.NAME,'city'))
city.select_by_visible_text('臺中市')
random_delay()
region_btn = Select(driver.find_element(By.NAME,'town'))
region_btn.select_by_visible_text('北區')
btn='#main_form > div:nth-child(2) > div.form-group.mt-0.form-check-inline.qry_general > div > font:nth-child(1) > a'
button = driver.find_elements(By.CSS_SELECTOR, btn)
if (len(button)>0):
  print(button[0].text)
button[0].click()
random_delay()
#print(driver.page_source)
random_delay()
table = driver.find_element(By.XPATH, '//*[@id="price_table"]')
rows = table.find_elements(By.TAG_NAME, 'tr')

# 讀取表格內容
table_data = []
for row in rows:
  # 獲取每行中的所有單元格
  cells = row.find_elements(By.TAG_NAME, 'td')
  row_data = [cell.text for cell in cells]  # 獲取每個單元格的文字
  if row_data:  # 過濾掉空的行
    print(row.get_attribute('outerHTML'))
    table_data.append(row_data)

# 打印表格資料
for row in table_data:
    print(row)

#print(table)
driver.quit()


搜尋
<tr role="row" class="table-text table-odd"><td class=" details-control" tabindex="0"><span style="display:none;">北區忠太東路０１１９號２１樓之１４</span><a class="a_link ellipsis" href="#row_3" id="3">北區忠太東路１１９號２１樓之１４<img src="../images/info_ico.png" width="18px" height="18px"></a></td><td>麗園道</td><td>136</td><td class="sorting_1">113/10/16</td><td>14.9</td><td>9.14</td><td>50.13</td><td><span style="display:none;">bt05</span><span class="bt05_br"></span></td><td>29</td><td style="display: none;">二十一層/二十一層</td><td style="display: none;">住家用</td><td style="display: none;"><span style="display:none;">1-1-0</span><span class="info_land">土</span>1 <span class="info_building">建</span>1<span class="info_garage">車</span>0</td><td style="display: none;">1房1衛</td><td style="display: none;"></td><td style="display: none;">有</td><td style="display: none;">有</td><td style="display: none;"><a href="javascript:void(0);"><button class="border_btn_a history">歷次</button></a><a href="javascript:void(0);"><button cl